# Implementing Aspect Based Sentiment Analysis using Python

Link : https://medium.com/analytics-vidhya/aspect-based-sentiment-analysis-a-practical-approach-8f51029bbc4a
·


In [2]:
cd ..

/Users/vitrac/Desktop/M2 HEC/05 - Quinten Healthcare/quinten_healthcare


In [3]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Standard data manipulation and analysis libraries
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Natural Language Processing (NLP) libraries
import nltk
import nltk.stem
import stanfordnlp
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# Text preprocessing and machine learning libraries
import stanza
from tqdm import tqdm
from transformers import pipeline

/Users/vitrac/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
stanza.download('en')
nltk.download('punkt')
nltk.download('wordnet')
stanfordnlp.download('en')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

2023-10-19 08:12:12 INFO: Downloading default packages for language: en (English) ...
2023-10-19 08:12:14 INFO: File exists: /Users/vitrac/stanza_resources/en/default.zip
2023-10-19 08:12:17 INFO: Finished downloading models and saved to /Users/vitrac/stanza_resources.
[nltk_data] Downloading package punkt to /Users/vitrac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vitrac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: /Users/vitrac/stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: /Users/vitrac/stanfordnlp_resources/en_ewt_models.zip


 57%|█████▋    | 134M/235M [00:23<00:17, 5.65MB/s] 

In [4]:
from src.data_processing.preprocessing import process_dataframe

file_path = 'data/raw_data_healthcare.csv'
df = pd.read_csv(file_path)
df = process_dataframe(df)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vitrac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vitrac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text_index,medication,rate,comment,cleaned_comment,Treatment name,Treatment code,Disease
0,0,inflectra infliximab for crohns disease,1.0,Insurance forced me to switch to inflectra due...,insurance force switch treatment_name due chea...,inflectra,infliximab,crohns disease
1,1,remicade infliximab for rheumatoid arthritis,NaN,My wife had RA from when she was a child. She ...,wife ra child use normal meds till treatment_n...,remicade,infliximab,rheumatoid arthritis
2,2,remicade infliximab for ulcerative colitis,1.0,This medication Remicade made me in constant p...,medication treatment_name make constant pain s...,remicade,infliximab,ulcerative colitis
3,3,inflectra infliximab for crohns disease,1.0,"I have Fistulizing Crohn's, dx'd 6 yrs ago @ 3...",fistulizing diseases dxd 6 yrs ago 36yo 11 sur...,inflectra,infliximab,crohns disease
4,4,remicade infliximab for ulcerative colitis,10.0,Remicade literally gave me my life back 9 yrs ...,treatment_name literally give life back 9 yrs ...,remicade,infliximab,ulcerative colitis


### 2 - Implementing Preprocessing for Aspect Based Sentiment Analysis 

In [ ]:
import re 
import nltk
import string 
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

def tokenize_and_tag(sentence):
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    return tagged_words

def create_flagged_comment(taggedList):
    # Pour l'étape 3 - get taggedList
    newwordList = []
    flag = 0
    for i in range(0,len(taggedList)-1):
        if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
            newwordList.append(taggedList[i][0]+taggedList[i+1][0])
            flag=1
        else:
            if(flag==1):
                flag=0
                continue
            newwordList.append(taggedList[i][0])
            if(i==len(taggedList)-2):
                newwordList.append(taggedList[i+1][0])
    finaltxt = ' '.join(word for word in newwordList)
    return finaltxt

def create_flagged_comment_get_newwordList(taggedList):
    # Pour l'étape 3 - get newwordList
    newwordList = []
    flag = 0
    for i in range(0,len(taggedList)-1):
        if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
            newwordList.append(taggedList[i][0]+taggedList[i+1][0])
            flag=1
        else:
            if(flag==1):
                flag=0
                continue
            newwordList.append(taggedList[i][0])
            if(i==len(taggedList)-2):
                newwordList.append(taggedList[i+1][0])
    return newwordList

def tokenize_and_pos_tag(comment):
    # Pour l'étape 4 - supprime les stopwords 
    stop_words = set(stopwords.words('english'))
    new_txt_list = nltk.word_tokenize(comment)
    wordsList = [w for w in new_txt_list if not w in stop_words]
    taggedList = nltk.pos_tag(wordsList)
    return taggedList

nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English

def extract_dependencies(row):
    # Fonction pour extraire les dépendances syntaxiques
    # Pour l'étape 5
    sentence = row['flagged_comment']
    newwordList = row['newwordList']
    doc = nlp(sentence)
    dep_node = []
    for dep_edge in doc.sentences[0].dependencies:
        dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
    for i in range(0, len(dep_node)):
        if int(dep_node[i][1]) != 0:
            source_index = int(dep_node[i][1]) - 1
            if 0 <= source_index < len(newwordList):
                dep_node[i][1] = newwordList[source_index]
            else:
                dep_node[i][1] = 'Out of range'
    return dep_node

def extract_features(df):
    df = df.copy()
    # Etape 5
    featureList = []  # To store features for each row
    categoriesList = []  # To store categories for each row
    
    for tagged_comment in df['tagged_comment']:
        features = []
        categories = []
        
        for word, pos in tagged_comment:
            if pos in ['JJ', 'NN', 'JJR', 'NNS', 'RB']:
                features.append([word, pos])
                categories.append(pos)
        
        featureList.append(features)
        categoriesList.append(categories)
    
    df['featureList'] = featureList
    df['categoriesList'] = categoriesList
    return df

def preprocessing_for_ABSA(df):

    # Replace disease, treatment code and treatment name per generic formulas
    replacement_dict = {
        # Disease replacements 
        'crohns disease': 'disease',
        'rheumatoid arthritis': 'disease',
        'ulcerative colitis': 'disease',
        'crohns disease maintenance': 'disease',
        'psoriatic arthritis': 'disease',
        'ankylosing spondylitis': 'disease',
        'ulcerative colitis maintenance': 'disease',
        'crohn': 'disease',
        "chron's": 'disease',
        'crohns': 'disease',
        'Crohns disease': 'disease',
        'Rheumatoid arthritis': 'disease',
        'Rheumatoid Arthritis': 'disease',
        'Ulcerative colitis': 'disease',
        'Ulcerative Colitis': 'disease',
        'Crohns disease maintenance': 'disease',
        'Psoriatic arthritis': 'disease',
        'Psoriatic Arthritis': 'disease',
        'Ankylosing Spondylitis': 'disease',
        'Ankylosing spondylitis': 'disease',
        'Ulcerative colitis maintenance': 'disease',
        "Chron's": 'disease',
        'Crohns': 'disease',
        'Crohn': 'disease',
        'RA': 'disease',
        
        # Treatment code replacements
        'infliximab': 'treatment',
        'adalimumab': 'treatment',
        'certolizumab': 'treatment',
        'golimumab': 'treatment',
        'aria golimumab': 'treatment',
        'vedolizumab': 'treatment',
        'ustekinumab': 'treatment',
        'natalizumab': 'treatment',
        'Infliximab': 'treatment',
        'Adalimumab': 'treatment',
        'Certolizumab': 'treatment',
        'Golimumab': 'treatment',
        'Aria golimumab': 'treatment',
        'Vedolizumab': 'treatment',
        'Ustekinumab': 'treatment',
        'Natalizumab': 'treatment',

        # Treatment name replacements
        'inflectra': 'treatment',
        'remicade': 'treatment',
        'renflexis': 'treatment',
        'humira': 'treatment',
        'cimzia': 'treatment',
        'simponi': 'treatment',
        'entyvio': 'treatment',
        'stelara': 'treatment',
        'tysabri': 'treatment',
        'Inflectra': 'treatment',
        'Remicade': 'treatment',
        'Renflexis': 'treatment',
        'Humira': 'treatment',
        'Cimzia': 'treatment',
        'Simponi': 'treatment',
        'Entyvio': 'treatment',
        'Stelara': 'treatment',
        'Tysabri': 'treatment'
    }
    
    # Appliquez les remplacements dans la colonne 'cleaned_comment' en utilisant le dictionnaire
    df['tokenized_comments'] = df['comment'].replace(replacement_dict, regex=True)
    
    # Étape 1: Tokeniser les commentaires en phrases
    df['tokenized_comments'] = df['tokenized_comments'].str.lower()
    df['tokenized_comments'] = df['tokenized_comments'].str.replace('side effect', 'sideeffect')
    df['tokenized_comments'] = df['tokenized_comments'].str.replace(f"[{re.escape(string.punctuation)}]", "", regex=True)
    df['tokenized_comments'] = df['tokenized_comments'].apply(nltk.sent_tokenize)
    print('Step 1 : Done')

    # Étape 2: Tokenize each sentence into words, et tag each element - 
    df['tagged_comment'] = df['tokenized_comments'].apply(lambda sentences: [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentences])
    df['tagged_comment'] = df['tagged_comment'].apply(lambda sentences: sentences[0])
    print('Step 2 : Done')

    # Étape 3: Appliquer la fonction create_flagged_comment à chaque ligne
    df['flagged_comment'] = df['tagged_comment'].apply(create_flagged_comment)
    df['newwordList'] = df['tagged_comment'].apply(create_flagged_comment_get_newwordList)
    print('Step 3 : Done')

    # Etape 4 : Appliquer la fonction pour créer la nouvelle colonne 'new_sentence'
    df['new_sentence'] = df['flagged_comment'].apply(tokenize_and_pos_tag)
    print('Step 4 : Done')

    # Etape 5 : Appliquer la fonction pour créer la nouvelle colonne 'dep_node'
    tqdm.pandas(desc="Extracting dependencies")
    df['dep_node'] = df.progress_apply(extract_dependencies, axis=1)
    print('Step 5 : Done')

    # Etape 6 : Extracting the features
    # We select only those sublists from the <dep_node> that could probably contain the features.
    df = extract_features(df)
    print('Step 6 : Done')

    return df

2023-10-17 18:19:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-17 18:19:58 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-10-17 18:19:58 INFO: Using device: cpu
2023-10-17 18:19:58 INFO: Loading: tokenize
2023-10-17 18:19:58 INFO: Loading: pos
2023-10-17 18:19:58 INFO: Loading: lemma
2023-10-17 18:19:58 INFO: Loading: constituency
2023-10-17 18:19:59 INFO: Loading: depparse
2023-10-17 18:19:59 INFO: Loading: sentiment
2023-10-17 18:19:59 INFO: Loading: ner
2023-10-17 18:1

In [ ]:
# Preprocess the positive dataframe
df_sentiment_based = df.copy()
df_sentiment_based = preprocessing_for_ABSA(df_sentiment_based)
df_sentiment_based.head()

Step 1 : Done
Step 2 : Done
Step 3 : Done
Step 4 : Done


Extracting dependencies: 100%|██████████| 784/784 [35:28<00:00,  2.71s/it] 

Step 5 : Done
Step 6 : Done


,text_index,medication,rate,comment,cleaned_comment,Treatment name,Treatment code,Disease,sentiment,sentiment_score,tokenized_comments,tagged_comment,flagged_comment,newwordList,new_sentence,dep_node,featureList,categoriesList
0,0,inflectra infliximab for crohns disease,1.0,Insurance forced me to switch to inflectra due...,insurance force switch treatment_name due chea...,inflectra,infliximab,crohns disease,NEGATIVE,0.978987,[insurance forced me to switch to treatment du...,"[(insurance, NN), (forced, VBD), (me, PRP), (t...",insurance forced me to switch to treatment due...,"[insurance, forced, me, to, switch, to, treatm...","[(insurance, NN), (forced, VBD), (switch, NN),...","[[insurance, forced, nsubj], [forced, 0, root]...","[[insurance, NN], [treatment, NN], [due, JJ], ...","[NN, NN, JJ, JJR, NNS, RB, NN, NN, NN, NNS, NN..."
1,1,remicade infliximab for rheumatoid arthritis,NaN,My wife had RA from when she was a child. She ...,wife ra child use normal med till treatment_na...,remicade,infliximab,rheumatoid arthritis,NEGATIVE,0.993057,[my wife had disease from when she was a child...,"[(my, PRP$), (wife, NN), (had, VBD), (disease,...",my wife had disease from when she was a child ...,"[my, wife, had, disease, from, when, she, was,...","[(wife, NN), (disease, NN), (child, NN), (used...","[[my, wife, nmod:poss], [wife, had, nsubj], [h...","[[wife, NN], [disease, NN], [child, NN], [norm...","[NN, NN, NN, JJ, NNS, NN, NN, NN, JJ, JJ, NN, ..."
2,2,remicade infliximab for ulcerative colitis,1.0,This medication Remicade made me in constant p...,medication treatment_name make constant pain s...,remicade,infliximab,ulcerative colitis,NEGATIVE,0.995946,[this medication treatment made me in constant...,"[(this, DT), (medication, NN), (treatment, NN)...",this medicationtreatment made me in constant p...,"[this, medicationtreatment, made, me, in, cons...","[(medicationtreatment, NN), (made, VBD), (cons...","[[this, medicationtreatment, det], [medication...","[[medication, NN], [treatment, NN], [constant,...","[NN, NN, JJ, NN, NN, NNS, NNS, NN, NN, RB, JJ,..."
3,3,inflectra infliximab for crohns disease,1.0,"I have Fistulizing Crohn's, dx'd 6 yrs ago @ 3...",fistulizing disease dxd 6 yr ago 36yo 11 surge...,inflectra,infliximab,crohns disease,NEGATIVE,0.993900,[i have fistulizing diseases dxd 6 yrs ago 36...,"[(i, NNS), (have, VBP), (fistulizing, VBG), (d...",i have fistulizing diseases dxd 6 yrs ago 36yo...,"[i, have, fistulizing, diseases, dxd, 6, yrs, ...","[(fistulizing, VBG), (diseases, NNS), (dxd, VB...","[[i, have, nsubj], [have, 0, root], [fistulizi...","[[i, NNS], [diseases, NNS], [yrs, NNS], [ago, ...","[NNS, NNS, NNS, RB, NNS, RB, JJ, NNS, NNS, NN,..."
4,4,remicade infliximab for ulcerative colitis,10.0,Remicade literally gave me my life back 9 yrs ...,treatment_name literally give life back 9 yr a...,remicade,infliximab,ulcerative colitis,NEGATIVE,0.994059,[treatment literally gave me my life back 9 yr...,"[(treatment, NN), (literally, RB), (gave, VBD)...",treatment literally gave me my life back 9 yrs...,"[treatment, literally, gave, me, my, life, bac...","[(treatment, NN), (literally, RB), (gave, VBD)...","[[treatment, gave, nsubj], [literally, gave, a...","[[treatment, NN], [literally, RB], [life, NN],...","[NN, RB, NN, RB, NN, NN, RB, JJ, NNS, NNS, NNS..."


In [ ]:
from collections import Counter

def extract_final_clusters(df):
    
    def create_feature_clusters(df):
        feature_clusters = []
        full_feature_list = []

        for index, row in df.iterrows():
            dep_node = row['dep_node']
            features = row['featureList']
            full_feature_list.append(features)
            for feature, pos in features:
                feature_cluster = []
                for j in dep_node:
                    if (j[0] == feature or j[1] == feature) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]):
                        if j[0] == feature:
                            feature_cluster.append(j[1])
                        else:
                            feature_cluster.append(j[0])
                if feature_cluster:
                    feature_clusters.append([feature, feature_cluster])
        # Applatir la liste full_feature_list
        flattened_feature_list = [item for sublist in full_feature_list for item in sublist]
        return feature_clusters, flattened_feature_list

    feature_clusters, full_feature_list = create_feature_clusters(df)

    finalcluster = []
    dic = {}
    for i in full_feature_list:
        dic[i[0]] = i[1]
    for i in feature_clusters:
        if dic[i[0]] == "NN":
            finalcluster.append(i)
    finalcluster = [item for item in finalcluster if item[0] not in ['i', 'say', 'wife', 'child', 'yrs', 'year', 't', 'omg', ]]
    finalcluster = [item for item in finalcluster if len(item[1]) < 4]

    return finalcluster

def extract_sentiments(df_preprocessed):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Final cluster for positive sentiments
    finalcluster = extract_final_clusters(df_preprocessed)
    all_sentiments = [sentiment[0] for sentiment in finalcluster]
    sentiment_counts = Counter(all_sentiments)
    sorted_sentiments = sentiment_counts.most_common()
    sorted_sentiments = [[sentiment, count] for sentiment, count in sorted_sentiments]

    for i in range(len(sorted_sentiments)):
        sentiment = sorted_sentiments[i][0]
        mots_associes = []
        for j in range(len(finalcluster)):
            if sentiment == finalcluster[j][0]:
                mots_associes.append(finalcluster[j][1])
        mots_associes = [item for sublist in mots_associes for item in sublist]
        mots_associes_lemmatized = [lemmatizer.lemmatize(word, 'v') for word in mots_associes]
        mots_associes_lemmatized = [word for word in mots_associes_lemmatized if word not in stop_words]
        mots_associes_lemmatized = list(set(mots_associes_lemmatized))
        mots_associes_lemmatized = [word[:-1] if word.endswith('i') else word for word in mots_associes_lemmatized]
        sorted_sentiments[i].append(mots_associes_lemmatized)

    return sorted_sentiments

In [ ]:
# Save the output
csv_file_path = 'data/sorted_feelings.csv'
sorted_sentiments = extract_sentiments(df_sentiment_based)
sorted_sentiments.to_csv(csv_file_path, index=False, encoding='utf-8')
print(f'DataFrame has been saved to {csv_file_path}')

In [5]:
from src.plots.Sentiment_analysis import create_tree_map

df_sorted_sentiments = pd.read_csv('data/sorted_feelings.csv')
create_tree_map(df_sorted_sentiments)

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/vitrac/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [14]:
import warnings
warnings.filterwarnings("ignore")
from src.plots.Sentiment_analysis import show_treatment_treemap

topics = ['treatment', 'pain', 'body', 'skin', 'fatigue', 'hair', 'sideeffect', 'weight', 'energy', 'legs']
for topic in topics:
    show_treatment_treemap(df_sorted_sentiments, sentiment_filter=topic)
